<a href="https://colab.research.google.com/github/Rajeswari-33/customer-support-ticket-triage/blob/main/Week1_Customer_Support_Triage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import re
from datetime import timedelta

In [11]:
df = pd.read_csv('customer_support_tickets.csv')

In [12]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_message'] = df['Ticket Description'].apply(clean_text)


In [13]:
def classify_issue(text):
    if any(word in text for word in ['payment', 'card', 'billing', 'charged']):
        return 'PAYMENT'
    elif any(word in text for word in ['login', 'password', 'signin', 'otp']):
        return 'LOGIN'
    elif any(word in text for word in ['delivery', 'shipping', 'delay', 'courier']):
        return 'DELIVERY'
    elif any(word in text for word in ['refund', 'return', 'money back']):
        return 'REFUND'
    elif any(word in text for word in ['bug', 'error', 'crash', 'issue']):
        return 'BUG'
    else:
        return 'GENERAL'

df['issue_type'] = df['clean_message'].apply(classify_issue)

In [14]:
def assign_priority(text):
    if any(word in text for word in ['urgent', 'critical', 'immediately', 'asap']):
        return 'P0'
    elif any(word in text for word in ['refund', 'failed', 'not working']):
        return 'P1'
    elif any(word in text for word in ['delay', 'slow', 'waiting']):
        return 'P2'
    else:
        return 'P3'

df['priority'] = df['clean_message'].apply(assign_priority)


In [15]:
sla_mapping = {'P0': 2, 'P1': 6, 'P2': 12, 'P3': 24}
df['sla_hours'] = df['priority'].map(sla_mapping)

# Convert First Response Time to datetime
df['First Response Time'] = pd.to_datetime(df['First Response Time'], errors='coerce')

df['sla_due_time'] = df['First Response Time'] + df['sla_hours'].apply(lambda x: timedelta(hours=x))


In [16]:
manager_report = df.groupby(['issue_type', 'priority']).size().reset_index(name='ticket_count')

print("=== Support Manager Report ===")
print(manager_report)


=== Support Manager Report ===
   issue_type priority  ticket_count
0         BUG       P0           131
1         BUG       P1            26
2         BUG       P2             6
3         BUG       P3          6933
4    DELIVERY       P1            12
5    DELIVERY       P2             5
6    DELIVERY       P3            92
7     GENERAL       P0            66
8     GENERAL       P1             3
9     GENERAL       P3           284
10      LOGIN       P0             3
11      LOGIN       P1           179
12      LOGIN       P3           260
13    PAYMENT       P0             4
14    PAYMENT       P1            20
15    PAYMENT       P3           205
16     REFUND       P0             8
17     REFUND       P1           128
18     REFUND       P3           104


In [17]:
output_file = 'final_ticket_triage_output.csv'
df.to_csv(output_file, index=False)

print("\n✅ Project completed successfully")
print("📁 Output file generated:", output_file)


✅ Project completed successfully
📁 Output file generated: final_ticket_triage_output.csv
